In [1]:
import numpy as np
import math
from skimage.transform import resize
import scipy.ndimage
import matplotlib.pyplot as plt

# Downsample using order 8 Chebyshev type I filter
def downsample(data):
    return scipy.signal.decimate(data, 6, axis = 0)

def prepare_data(mat):
    # Left footstep
    dataL = mat['dataL']
    # Right footstep
    dataR = mat['dataR']  
    return (dataL, dataR)

def calGRF(data):
    return np.add.accumulate(data)

def process_foot(data):
    Tmax = 1600
    data = data[:Tmax]
    data = downsample(data)
    grf = calGRF(data)
    s_avg = data.mean(axis=1)
    s_upper = np.amax(data,axis=1)
    s_lower = np.amin(data,axis=1)
    grf_t = grf.mean(axis=1)
    grf_t_max = np.max(grf_t)
    grf_t /= grf_t_max
    processed = np.append(s_avg, [grf_t,s_upper,s_lower])
    return processed 

def process_matlab(mat):
    dataL, dataR = prepare_data(mat)
    footL, footR = process_foot(dataL), process_foot(dataR)
    processed = np.append(footL,footR)
    return processed


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import scipy.io
import scipy.signal

In [3]:
index_files_path = 'SFootBD/IndexFiles/'
data_files_path = 'SFootBD/SFootBD/'

train_list_file = index_files_path + 'LstTrain'
imposter_list_file = index_files_path + 'LstImp'
test_evaluation_list_file = index_files_path + 'LstTestEvaluation'
test_validation_list_file = index_files_path + 'LstTestValidation'

In [4]:
def load_features(filename):
    mat_file = os.path.join(data_files_path, filename + '.mat')
    mat = scipy.io.loadmat(mat_file)
    return process_matlab(mat)

def load_data_file(index_file):
    return pd.read_csv(index_file, sep = ' ', header = None)

def load_dataset(data_list, clients):    
    X = []
    y = []
    
    for item in data_list.itertuples():
        X_item = load_features(item[2])
        y_item = item[1]
        
        matches = np.where(clients == y_item)[0]
        
        X.append(X_item)
        # clients.shape[0] becomes imposter class
        y.append(matches[0] if matches.shape[0] != 0 else clients.shape[0])
        
    return np.array(X), np.array(y)

In [ ]:
train_index = load_data_file(train_list_file)

clients = np.unique(train_index[0])

X_train, y_train = load_dataset(train_index, clients)

In [ ]:
imposter_index = load_data_file(imposter_list_file)

X_imposter, y_imposter = load_dataset(imposter_index, clients)

X_train = np.append(X_train, X_imposter, axis = 0)
y_train = np.append(y_train, y_imposter, axis = 0)

In [7]:
# Validation set is too big
val_index = load_data_file(test_validation_list_file)

X_val, y_val = load_dataset(val_index, clients)

X_train = np.append(X_val, X_imposter, axis = 0)
y_train = np.append(y_val, y_imposter, axis = 0)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

model = Pipeline([
    ("std_scaler", StandardScaler()),
    #("model", OneVsRestClassifier(SVC(kernel='rbf')))
    ("model", SVC(kernel='rbf'))
])

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [9]:
# Validation set is too big
eval_index = load_data_file(test_evaluation_list_file)

X_test, y_test = load_dataset(eval_index, clients)

In [13]:
y_pred = model.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7072727272727273

In [15]:
#import bob.measure #doesn't work on windows

ModuleNotFoundError: No module named 'bob'